In [52]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

# Here, we just set how many documents we'll process for automatic testing- you can safely ignore this!
n_docs = 500 if 'CI' in os.environ else 2591

from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

train_cands = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev_cands   = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test_cands  = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()
print(len(dev_cands))

2814


In [53]:
from util import load_external_labels

# %time load_external_labels(session, Spouse, annotator_name='gold')

from snorkel.annotations import load_gold_labels

#L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
#L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)

# L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
# L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
# gold_labels_dev = [L[0,0] if L[0,0]==1 else -1 for L in L_gold_dev]
gold_labels_dev = [L[0,0] for L in L_gold_dev]


from snorkel.learning.utils import MentionScorer

Instructions for updating:
Use the retry module or similar alternatives.


In [54]:
#gold_labels_dev = [x[0,0] for x in L_gold_dev.todense()]
#for i,L in enumerate(gold_labels_dev):
#    print(i,gold_labels_dev[i])

# gold_labels_dev = []
# for i,L in enumerate(L_gold_dev):
#     gold_labels_dev.append(L[0,0])
    
# gold_labels_test = []
# for i,L in enumerate(L_gold_test):
#     gold_labels_test.append(L[0,0])
    
# print(len(gold_labels_dev),len(gold_labels_test))
# print(gold_labels_dev.count(1),gold_labels_dev.count(-1))
# print(len(gold_labels_dev))

print(gold_labels_dev.count(1),gold_labels_dev.count(0))
print(len(gold_labels_dev))

189 2625
2814


In [96]:
from gensim.parsing.preprocessing import STOPWORDS
import gensim.matutils as gm

from gensim.models.keyedvectors import KeyedVectors

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('../../../snorkel/tutorials/glove_w2v.txt', binary=False)  # C binary format


wordvec_unavailable= set()
def write_to_file(wordvec_unavailable):
    with open("wordvec_unavailable.txt","w") as f:
        for word in wordvec_unavailable:
            f.write(word+"\n")

def preprocess(tokens):
    btw_words = [word for word in tokens if word not in STOPWORDS]
    btw_words = [word for word in btw_words if word.isalpha()]
    return btw_words

def get_word_vectors(btw_words): # returns vector of embeddings of words
    word_vectors= []
    for word in btw_words:
        try:
            word_v = np.array(model[word])
            word_v = word_v.reshape(len(word_v),1)
            #print(word_v.shape)
            word_vectors.append(model[word])
        except:
            wordvec_unavailable.add(word)
    return word_vectors

def get_similarity(word_vectors,target_word): # sent(list of word vecs) to word similarity
    similarity = 0
    target_word_vector = 0
    try:
        target_word_vector = model[target_word]
    except:
        wordvec_unavailable.add(target_word+" t")
        return similarity
    target_word_sparse = gm.any2sparse(target_word_vector,eps=1e-09)
    for wv in word_vectors:
        wv_sparse = gm.any2sparse(wv, eps=1e-09)
        similarity = max(similarity,gm.cossim(wv_sparse,target_word_sparse))
    return similarity


In [57]:
####### Discrete ##########

spouses = {'spouse', 'wife', 'husband', 'ex-wife', 'ex-husband'}
family = {'father', 'mother', 'sister', 'brother', 'son', 'daughter',
              'grandfather', 'grandmother', 'uncle', 'aunt', 'cousin'}
family = family | {f + '-in-law' for f in family}
other = {'boyfriend', 'girlfriend' 'boss', 'employee', 'secretary', 'co-worker'}

# Helper function to get last name
def last_name(s):
    name_parts = s.split(' ')
    return name_parts[-1] if len(name_parts) > 1 else None    

def LF_husband_wife(c):
    return (1,1) if len(spouses.intersection(get_between_tokens(c))) > 0 else (0,0)

def LF_husband_wife_left_window(c):
    if len(spouses.intersection(get_left_tokens(c[0], window=2))) > 0:
        return (1,1)
    elif len(spouses.intersection(get_left_tokens(c[1], window=2))) > 0:
        return (1,1)
    else:
        return (0,0)
    
def LF_same_last_name(c):
    p1_last_name = last_name(c.person1.get_span())
    p2_last_name = last_name(c.person2.get_span())
    if p1_last_name and p2_last_name and p1_last_name == p2_last_name:
        if c.person1.get_span() != c.person2.get_span():
            return (1,1)
    return (0,0)

def LF_no_spouse_in_sentence(c):
    return (-1,1) if np.random.rand() < 0.75 and len(spouses.intersection(c.get_parent().words)) == 0 else (0,0)

def LF_and_married(c):
    return (1,1) if 'and' in get_between_tokens(c) and 'married' in get_right_tokens(c) else (0,0)
    
def LF_familial_relationship(c):
    return (-1,1) if len(family.intersection(get_between_tokens(c))) > 0 else (0,0)

def LF_family_left_window(c):
    if len(family.intersection(get_left_tokens(c[0], window=2))) > 0:
        return (-1,1)
    elif len(family.intersection(get_left_tokens(c[1], window=2))) > 0:
        return (-1,1)
    else:
        return (0,0)

def LF_other_relationship(c):
    return (-1,1) if len(other.intersection(get_between_tokens(c))) > 0 else (0,0)


import bz2

# Function to remove special characters from text
def strip_special(s):
    return ''.join(c for c in s if ord(c) < 128)

# Read in known spouse pairs and save as set of tuples
with bz2.BZ2File('data/spouses_dbpedia.csv.bz2', 'rb') as f:
    known_spouses = set(
        tuple(strip_special(x.decode('utf-8')).strip().split(',')) for x in f.readlines()
    )
# Last name pairs for known spouses
last_names = set([(last_name(x), last_name(y)) for x, y in known_spouses if last_name(x) and last_name(y)])
    
def LF_distant_supervision(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    return (1,1) if (p1, p2) in known_spouses or (p2, p1) in known_spouses else (0,0)

def LF_distant_supervision_last_names(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    p1n, p2n = last_name(p1), last_name(p2)
    return (1,1) if (p1 != p2) and ((p1n, p2n) in last_names or (p2n, p1n) in last_names) else (0,0)


LFs = [
    LF_distant_supervision, LF_distant_supervision_last_names, 
    LF_husband_wife, LF_husband_wife_left_window, LF_same_last_name,
    LF_no_spouse_in_sentence, LF_and_married, LF_familial_relationship, 
    LF_family_left_window, LF_other_relationship
]

In [92]:
# ##### Continuous ################


# import re
# from snorkel.lf_helpers import (
#     get_left_tokens, get_right_tokens, get_between_tokens,
#     get_text_between, get_tagged_text,
# )


# spouses = {'spouse', 'wife', 'husband', 'ex-wife', 'ex-husband'}
# family = {'father', 'mother', 'sister', 'brother', 'son', 'daughter',
#               'grandfather', 'grandmother', 'uncle', 'aunt', 'cousin'}
# family = family | {f + '-in-law' for f in family}
# other = {'boyfriend', 'girlfriend' 'boss', 'employee', 'secretary', 'co-worker'}

# # Helper function to get last name
# def last_name(s):
#     name_parts = s.split(' ')
#     return name_parts[-1] if len(name_parts) > 1 else None    

# def LF_husband_wife(c):
#     global LF_Threshold
#     sc = 0
#     word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
#     for sw in spouses:
#         sc=max(sc,get_similarity(word_vectors,sw))
#     return (1,sc)

# def LF_husband_wife_left_window(c):
#     global LF_Threshold
#     sc_1 = 0
#     word_vectors = get_word_vectors(preprocess(get_left_tokens(c[0])))
#     for sw in spouses:
#         sc_1=max(sc_1,get_similarity(word_vectors,sw))
        
#     sc_2 = 0
#     word_vectors = get_word_vectors(preprocess(get_left_tokens(c[1])))
#     for sw in spouses:
#         sc_2=max(sc_2,get_similarity(word_vectors,sw))
#     return(1,max(sc_1,sc_2))
    
# def LF_same_last_name(c):
#     p1_last_name = last_name(c.person1.get_span())
#     p2_last_name = last_name(c.person2.get_span())
#     if p1_last_name and p2_last_name and p1_last_name == p2_last_name:
#         if c.person1.get_span() != c.person2.get_span():
#             return (1,1)
#     return (0,0)

# def LF_no_spouse_in_sentence(c):
#     return (-1,0.75) if np.random.rand() < 0.75 and len(spouses.intersection(c.get_parent().words)) == 0 else (0,0)

# def LF_and_married(c):
#     global LF_Threshold
#     word_vectors = get_word_vectors(preprocess(get_right_tokens(c)))
#     sc = get_similarity(word_vectors,'married')
    
#     if 'and' in get_between_tokens(c):
#         return (1,sc)
#     else:
#         return (0,0)

# def LF_familial_relationship(c):
#     sc = 0
#     word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
#     for fw in family:
#         sc=max(sc,get_similarity(word_vectors,fw))
        
#     return (-1,sc) 

# def LF_family_left_window(c):
#     sc_1 = 0
#     word_vectors = get_word_vectors(preprocess(get_left_tokens(c[0])))
#     for fw in family:
#         sc_1=max(sc_1,get_similarity(word_vectors,fw))
        
#     sc_2 = 0
#     word_vectors = get_word_vectors(preprocess(get_left_tokens(c[1])))
#     for fw in family:
#         sc_2=max(sc_2,get_similarity(word_vectors,fw))
        
#     return (-1,max(sc_1,sc_2))

# def LF_other_relationship(c):
#     sc = 0
#     word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
#     for ow in other:
#         sc=max(sc,get_similarity(word_vectors,ow))
        
#     return (-1,sc) 

# # def LF_other_relationship_left_window(c):
# #     sc = 0
# #     word_vectors = get_word_vectors(preprocess(get_left_tokens(c)))
# #     for ow in other:
# #         sc=max(sc,get_similarity(word_vectors,ow))
# #     return (-1,sc) 

# import bz2

# # Function to remove special characters from text
# def strip_special(s):
#     return ''.join(c for c in s if ord(c) < 128)

# # # Read in known spouse pairs and save as set of tuples
# # with bz2.BZ2File('data/spouses_dbpedia.csv.bz2', 'rb') as f:
# #     known_spouses = set(
# #         tuple(strip_special(x).strip().split(',')) for x in f.readlines()
# #     )
# # # Last name pairs for known spouses
# # last_names = set([(last_name(x), last_name(y)) for x, y in known_spouses if last_name(x) and last_name(y)])
    
# def LF_distant_supervision(c):
#     p1, p2 = c.person1.get_span(), c.person2.get_span()
#     return (1,1) if (p1, p2) in known_spouses or (p2, p1) in known_spouses else (0,0)

# def LF_distant_supervision_last_names(c):
#     p1, p2 = c.person1.get_span(), c.person2.get_span()
#     p1n, p2n = last_name(p1), last_name(p2)
#     return (1,1) if (p1 != p2) and ((p1n, p2n) in last_names or (p2n, p1n) in last_names) else (0,1)

# import numpy as np

# # def LF_Three_Lists_Left_Window(c):
# #     global softmax_Threshold
# #     c1,s1 = LF_husband_wife_left_window(c)
# #     c2,s2 = LF_family_left_window(c)
# #     c3,s3 = LF_other_relationship_left_window(c)
# #     sc = np.array([s1,s2,s3])
# #     c = [c1,c2,c3]
# #     sharp_param = 1.5
# #     prob_sc = np.exp(sc * sharp_param - np.max(sc))
# #     prob_sc = prob_sc / np.sum(prob_sc)
# #     #print 'Left:',s1,s2,s3,prob_sc
    
# #     if s1==s2 or s3==s1:
# #         return (0,0)
# #     return c[np.argmax(prob_sc)],1

# # def LF_Three_Lists_Between_Words(c):
# #     global softmax_Threshold
# #     c1,s1 = LF_husband_wife(c)
# #     c2,s2 = LF_familial_relationship(c)
# #     c3,s3 = LF_other_relationship(c)
# #     sc = np.array([s1,s2,s3])
# #     c = [c1,c2,c3]
# #     sharp_param = 1.5
    
# #     prob_sc = np.exp(sc * sharp_param - np.max(sc))
# #     prob_sc = prob_sc / np.sum(prob_sc)
# #     #print 'BW:',s1,s2,s3,prob_sc
# #     if s1==s2 or s3==s1:
# #         return (0,0)
# #     return c[np.argmax(prob_sc)],1
    
# LFs = [
#     LF_distant_supervision, LF_distant_supervision_last_names, 
#     LF_husband_wife, LF_husband_wife_left_window, LF_same_last_name,
#     LF_no_spouse_in_sentence, LF_and_married, LF_familial_relationship, 
#     LF_family_left_window, LF_other_relationship
# ]

In [5]:
''' output:

    [[[L_x1],[S_x1]],
     [[L_x2],[S_x2]],
     ......
     ......
    ]

'''
def get_L_S_Tensor(cands): 
    
    L_S = []
    for i,ci in enumerate(cands):
        L_S_ci=[]
        L=[]
        S=[]
        for LF in LFs:
            #print LF.__name__
            l,s = LF(ci)
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
        if(i%500==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
    return L_S


In [ ]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

dev_L_S = get_L_S_Tensor(dev_cands)

np.save("dev_L_S_smooth",np.array(dev_L_S))

train_L_S = get_L_S_Tensor(train_cands)
np.save("train_L_S_smooth",np.array(train_L_S))

print("--- %s seconds ---" % (time.time() - start_time))

# test_L_S = get_L_S_Tensor(test_cands)
# pkl.dump(test_L_S,open("test_L_S.p","wb"))

In [58]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

def draw2DArray(a):
    fig = plt.figure(figsize=(6, 3.2))
    ax = fig.add_subplot(111)
    ax.set_title('colorMap')
    plt.imshow(np.array(a))
    ax.set_aspect('equal')
    cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.patch.set_alpha(0)
    cax.set_frame_on(False)
    plt.colorbar(orientation='vertical')
    plt.show()
    
      
def report2dict(cr):
    # Parse rows
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)
    
    # Store in dictionary
    measures = tmp[0]

    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
    return pd.DataFrame(D_class_data).T

def predictAndPrint(pl):
    print("acc",accuracy_score(gold_labels_dev,pl))
#     print(precision_recall_fscore_support(true_labels,pl,average='macro'))
    print(confusion_matrix(gold_labels_dev,pl))
    draw2DArray(confusion_matrix(gold_labels_dev,pl))
    return report2dict(classification_report(gold_labels_dev, pl))# target_names=class_names))
    


def drawPRcurve(y_test,y_score,it_no):
    
    fig = plt.figure()
    splt = fig.add_subplot(111)
    

    precision, recall, _ = precision_recall_curve(y_test, y_score,pos_label=1)

    splt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    splt.fill_between(recall, precision, step='post', alpha=0.2,
                     color='b')
    average_precision = average_precision_score(y_test, y_score)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.05])
    plt.title('{0:d} Precision-Recall curve: AP={1:0.2f}'.format(it_no,
              average_precision))
    

def drawLossVsF1(y_loss,x_f1s,text,title):
    fig, ax = plt.subplots()
    ax.scatter(x_f1s, y_loss)

    plt.xlabel('f1-score')
    plt.ylabel('loss')
    plt.title(title)
    
    for i, txt in enumerate(text):
        ax.annotate(txt, (x_f1s[i],y_loss[i]))
        
    plt.savefig(title+".png")
    

In [119]:
LF_l = np.array([
    1,1,1,1,1,-1,1,-1,-1,-1
])

NoOfLFs= len(LFs)
NoOfClasses = 2
print(len(LFs),len(LF_l))

import numpy as np
# dev_L_S = np.load("dev_L_S_smooth.npy")
# train_L_S = np.load("train_L_S_smooth.npy")

dev_L_S = np.load("dev_L_S_discrete.npy")
train_L_S = np.load("train_L_S_discrete.npy")
print(dev_L_S.shape,train_L_S.shape)



10 10
(2814, 2, 10) (22276, 2, 10)


In [121]:
import random


def createSubsetOfDataAndLFs(train_L_S,dev_L_S,LF_l,data_subset_indices,LF_subset_indices):
    global NoOfLFs
    SizeOfDataSubSet = 10000
    NoOfLFsInLFSubset = len(LF_subset_indices)
    NoOfLFs = NoOfLFsInLFSubset
#     train_L_S = train_L_S[data_subset_indices,:]
    train_L_S = train_L_S[:,:,LF_subset_indices]
    print(data_subset_indices.shape,LF_subset_indices.shape)
    dev_L_S = dev_L_S[:,:,LF_subset_indices]
    LF_l = LF_l[LF_subset_indices]
    return train_L_S,dev_L_S,LF_l

def train_nl_penalties(LF_subset_indices,lr=0.1/len(train_L_S),it=5,th=tf.truncated_normal_initializer(0,0.1,seed)):
    global train_L_S,dev_L_S,LF_l
    print(LF_subset_indices)
    
    data_subset_indices = np.array(random.sample(range(train_L_S.shape[0]), SizeOfDataSubSet))

    train_L_S,dev_L_S,LF_l = createSubsetOfDataAndLFs(train_L_S,dev_L_S,LF_l,data_subset_indices,LF_subset_indices)
    print(train_L_S.shape,dev_L_S.shape,LF_l.shape)

    train_nl(lr,it,th) 

    train_nl_p(lr,it,th) 

    train_nl_p2(lr,it,th)

    train_nl_p3(lr,it,th) 

In [88]:
#call this only once for a kernel startup
from __future__ import absolute_import, division, print_function
seed = 12 
import tensorflow as tf
# BATCH_SIZE = 32

In [120]:
# Use full dataset and all LFs
LF_subset_indices = np.array([0,1,2,3,4,5,6,7,8,9])
train_nl_penalties(LF_subset_indices)

[0 1 2 3 4 5 6 7 8 9]
(10000,) (10,)
(22276, 2, 10) (2814, 2, 10) (10,)
0 loss 154550.32722203954
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[ 0.03625473 -0.09201922  0.04087745  0.03838884  0.04881964  0.07631783
   0.07385835  0.01965096  0.09410612 -0.01521934]]
{0: 2492, 1: 322}
(0.32298136645962733, 0.5502645502645502, 0.4070450097847358, None)

1 loss 154394.2097362968
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[ 0.02376233 -0.00917943  0.04839996  0.04515221  0.02658253  0.12737847
   0.02282324  0.01706548  0.05779753 -0.02330113]]
{0: 2486, 1: 328}
(0.32621951219512196, 0.5661375661375662, 0.413926499032882, None)

2 loss 154320.63792672456
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[ 0.02578081  0.02102967  0.05121235  0.04709643  0.02268402  0.18204747
   0.02531712

4 loss 154159.0631150702
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[ 4.14473160e-02  3.74993703e-02  7.16429435e-02  6.58696507e-02
   3.35547255e-02  2.89791883e-01  4.11253641e-02  2.11622138e-02
   2.15834607e-02 -1.72112189e-07]]
{0: 2463, 1: 351}
(0.30484330484330485, 0.5661375661375662, 0.3962962962962963, None)

[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[ 4.14473160e-02  3.74993703e-02  7.16429435e-02  6.58696507e-02
   3.35547255e-02  2.89791883e-01  4.11253641e-02  2.11622138e-02
   2.15834607e-02 -1.72112189e-07]]
Scores (Un-adjusted)
Pos. class accuracy: 0.566
Neg. class accuracy: 0.907
Precision            0.305
Recall               0.566
F1                   0.396
----------------------------------------
TP: 107 | FP: 244 | TN: 2381 | FN: 82

{0: 2463, 1: 351}
acc 0.8841506751954513
(array([0.96670727, 0.3048433 ]), array([0.907047

In [123]:
# Use full dataset and all LFs
LF_subset_indices = np.array([0,1,2,3,4,5,6,7,8,9])
train_nl_penalties(LF_subset_indices,th=tf.truncated_normal_initializer(1,0.1,seed) )

[0 1 2 3 4 5 6 7 8 9]
(10000,) (10,)
(22276, 2, 10) (2814, 2, 10) (10,)
0 loss 177328.68299044456
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[1.01874088 0.71637454 0.92309079 0.91201652 1.00877974 1.12112714
  1.07023119 1.08008369 1.18919429 1.12356771]]
{0: 2522, 1: 292}
(0.3527397260273973, 0.544973544973545, 0.4282744282744283, None)

1 loss 170637.539636164
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[0.92056097 0.61998249 0.84490993 0.83059571 0.91264682 1.19233795
  0.9717674  1.09133108 1.18678918 1.07515922]]
{0: 2522, 1: 292}
(0.3527397260273973, 0.544973544973545, 0.4282744282744283, None)

2 loss 166039.65982295244
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[0.82307524 0.5266278  0.7815422  0.76270372 0.81926292 1.25204292
  0.87374085 1.06482643 1.14503249 0.987396

pout Tensor("map_1/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("map_2/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t_k Tensor("mul_1:0", shape=(10,), dtype=float64)
zy Tensor("map_3/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("Log:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(), dtype=float64)
normloss Tensor("add_1:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
predict Tensor("ArgMax:0", shape=(?, 1), dtype=int64)
0 loss 245068.43749177322
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[1.02051008 0.72589385 0.95136584 0.93762739 1.01335664 1.12975561
  1.07147025 1.1126708  1.22644629 1.15909523]]
{0: 2522, 1: 292}
(0.3527397260273973, 0.544973544973545, 0.4282744282744283, None)

1 loss 240366.05178711584
[0.31754463 0.01421453 0.20955965

In [123]:
# Use full dataset and all LFs
LF_subset_indices = np.array([0,1,2,3,4,5,6,7,8,9])
train_nl_penalties(LF_subset_indices,th=tf.truncated_normal_initializer(1,0.1,seed) )

[0 1 2 3 4 5 6 7 8 9]
(10000,) (10,)
(22276, 2, 10) (2814, 2, 10) (10,)
0 loss 177328.68299044456
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[1.01874088 0.71637454 0.92309079 0.91201652 1.00877974 1.12112714
  1.07023119 1.08008369 1.18919429 1.12356771]]
{0: 2522, 1: 292}
(0.3527397260273973, 0.544973544973545, 0.4282744282744283, None)

1 loss 170637.539636164
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[0.92056097 0.61998249 0.84490993 0.83059571 0.91264682 1.19233795
  0.9717674  1.09133108 1.18678918 1.07515922]]
{0: 2522, 1: 292}
(0.3527397260273973, 0.544973544973545, 0.4282744282744283, None)

2 loss 166039.65982295244
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[0.82307524 0.5266278  0.7815422  0.76270372 0.81926292 1.25204292
  0.87374085 1.06482643 1.14503249 0.987396

pout Tensor("map_1/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("map_2/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t_k Tensor("mul_1:0", shape=(10,), dtype=float64)
zy Tensor("map_3/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("Log:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(), dtype=float64)
normloss Tensor("add_1:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
predict Tensor("ArgMax:0", shape=(?, 1), dtype=int64)
0 loss 245068.43749177322
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[1.02051008 0.72589385 0.95136584 0.93762739 1.01335664 1.12975561
  1.07147025 1.1126708  1.22644629 1.15909523]]
{0: 2522, 1: 292}
(0.3527397260273973, 0.544973544973545, 0.4282744282744283, None)

1 loss 240366.05178711584
[0.31754463 0.01421453 0.20955965

In [106]:
# for picking lfs randomly
# LF_subset_indices = np.array(random.sample(range(train_L_S.shape[2]), NoOfLFsInLFSubset))
LF_subset_indices = np.array([0,1,2,7,8,9])
train_nl_penalties(LF_subset_indices)

[0 1 2 7 8 9]
(10000,) (6,)
(10000, 2, 6) (2814, 2, 6) (6,)
0 loss 41641.76504616807
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[ 0.04529264 -0.09905075  0.03875814 -0.02447436  0.03313103 -0.01251145]]
{0: 2390, 1: 424}
(0.22641509433962265, 0.5079365079365079, 0.31321370309951063, None)

1 loss 41595.04026404794
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[-1.12811658e-04 -1.85190406e-02  1.06663502e-02 -5.73847509e-03
  -2.45000975e-03  3.99037312e-05]]
{0: 2217, 1: 597}
(0.1574539363484087, 0.4973544973544973, 0.23918575063613226, None)

2 loss 41588.887845803336
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[-2.36561654e-17 -4.50008643e-17 -4.28936544e-17  5.90739548e-17
   6.11807467e-17  2.60414069e-17]]
{0: 2814}
(0.0, 0.0, 0.0, None)



/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


3 loss 41588.830833630505
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[ 3.10903081e-17  4.82762452e-17 -2.59143103e-17  4.91687711e-18
  -2.88735145e-18  4.44713718e-17]]
{0: 2799, 1: 15}
(0.26666666666666666, 0.021164021164021163, 0.0392156862745098, None)

4 loss 41588.830833634966
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[ 6.66033802e-17  1.14761024e-16  6.06776279e-17 -4.18142729e-17
  -8.41424476e-17  3.50142153e-17]]
{0: 2202, 1: 612}
(0.1650326797385621, 0.5343915343915344, 0.25218476903870163, None)

[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[ 6.66033802e-17  1.14761024e-16  6.06776279e-17 -4.18142729e-17
  -8.41424476e-17  3.50142153e-17]]
Scores (Un-adjusted)
Pos. class accuracy: 0.0952
Neg. class accuracy: 0.909
Precision            0.07
Recall               0.0952
F1                   0.0807
----------------------------------------
TP: 18 | FP: 239 | TN: 2386 | FN: 171

{0: 2202, 1: 612}
acc 0.7

0 loss 81502.1506470213
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[ 0.21666989 -0.08647327  0.10881792  0.09944611  0.20484656  0.14030441]]
{0: 2568, 1: 246}
(0.37398373983739835, 0.48677248677248675, 0.4229885057471264, None)

1 loss 78833.59265509271
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[0.31546659 0.01260159 0.20749485 0.19653671 0.30100983 0.23885532]]
{0: 2562, 1: 252}
(0.373015873015873, 0.4973544973544973, 0.4263038548752835, None)

2 loss 76445.04474176883
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[0.41399234 0.11148472 0.30517841 0.29088138 0.39358324 0.33701439]]
{0: 2561, 1: 253}
(0.37549407114624506, 0.5026455026455027, 0.4298642533936652, None)

3 loss 74339.8757578947
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[0.51211409 0.2100796  0.40116541 0.38154931 0.48141777 0.43453633]]
{0: 2561, 1: 253}
(0.37549407114624506, 0.5026455026455027, 0.4298642533936652, None

In [108]:
LF_subset_indices = np.array([2,3,4,5,7,8])
train_nl_penalties(LF_subset_indices)

[2 3 4 5 7 8]
(10000,) (6,)
(10000, 2, 6) (2814, 2, 6) (6,)
0 loss 41638.92673303141
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[ 0.08449117 -0.11044035  0.02008056 -0.0598917   0.04483325  0.00326031]]
{0: 927, 1: 1887}
(0.041865394806571275, 0.41798941798941797, 0.07610789980732177, None)

1 loss 41588.54307785565
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[ 0.0130182  -0.05790362 -0.01682119 -0.11221936  0.01801132  0.00857004]]
{0: 981, 1: 1833}
(0.03600654664484452, 0.3492063492063492, 0.06528189910979229, None)

2 loss 41561.165261243885
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[-0.03361289 -0.05100757 -0.02642908 -0.16474845  0.01094241  0.00962209]]
{0: 1073, 1: 1741}
(0.031016657093624354, 0.2857142857142857, 0.055958549222797936, None)

3 loss 41534.27115888344
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[-0.05748832 -0.06126314 -0.03491478 -0.21682487  0.00836414  0.01079

pout Tensor("map_1/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 6), dtype=float64)
t_pout Tensor("map_2/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t_k Tensor("mul_1:0", shape=(6,), dtype=float64)
zy Tensor("map_3/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("Log:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(), dtype=float64)
normloss Tensor("add_1:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
predict Tensor("ArgMax:0", shape=(?, 1), dtype=int64)
0 loss 81470.2300283042
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[ 0.21656558 -0.08643751  0.10873977  0.10021223  0.20424723  0.13959845]]
{0: 2539, 1: 275}
(0.3381818181818182, 0.49206349206349204, 0.40086206896551724, None)

1 loss 78712.99664044699
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855]
[[0.31448979 0.01246084 0.20749486 0.19792972 0.2985

In [117]:
# Use full dataset all but no_spouse_in_sentence
LF_subset_indices = np.array([0,1,2,3,4,6,7,8,9])
train_nl_penalties(LF_subset_indices)

[0 1 2 3 4 6 7 8 9]
(10000,) (9,)
(22276, 2, 9) (2814, 2, 9) (9,)
0 loss 139171.781832511
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471]
[[ 0.05144432 -0.09275716  0.04617867  0.0424805   0.06674175  0.02729831
   0.1055682   0.00126014  0.05632088]]
{0: 2514, 1: 300}
(0.3233333333333333, 0.5132275132275133, 0.3967280163599182, None)

1 loss 138996.60347260014
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471]
[[ 0.00136959 -0.00913628  0.03101953  0.02755143  0.00709781  0.00101656
   0.05405614  0.00637396 -0.00092416]]
{0: 2481, 1: 333}
(0.3183183183183183, 0.5608465608465608, 0.40613026819923365, None)

2 loss 138967.96302785305
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471]
[[ 0.00068522  0.00072692  0.00771056  0.00653462  0.00065158  0.00057692
   0.0132444   0.00478876 -0.00060508]]
{0: 2475, 1: 339}
(0.3185840707964602

nls Tensor("mul:0", shape=(?, 9), dtype=float64)
pout Tensor("map_1/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 9), dtype=float64)
t_pout Tensor("map_2/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t_k Tensor("mul_1:0", shape=(9,), dtype=float64)
zy Tensor("map_3/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("Log:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(), dtype=float64)
normloss Tensor("add_1:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
predict Tensor("ArgMax:0", shape=(?, 1), dtype=int64)
0 loss 269085.54387308477
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471]
[[ 0.2163905  -0.08665923  0.10842293  0.09954735  0.2051548   0.1402791
   0.26774418  0.13053799  0.24643691]]
{0: 2510, 1: 304}
(0.3256578947368421, 0.5238095238095238, 0.40162271805273836, None)

1 loss 260535.2979800912

In [63]:
## normalized training with different params

def train_nl(lr,ep,th):
    
    BATCH_SIZE = 1
    tf.reset_default_graph()


    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

        labels = tf.convert_to_tensor(gold_labels_dev)
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)

        next_element = iterator.get_next()
#         print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                                 dtype=tf.float64)

    #     thetas = tf.get_variable('thetas', [1,NoOfLFs],\
    #                              initializer=tf.truncated_normal_initializer(1,0.1,seed),\
    #                              dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

#         print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
#         print("k",k)
#         print(alphas.graph)
#         print(thetas.graph)
        l,s =  tf.unstack(next_element,axis=1)
#         print(alphas)
#         print(s)
#         print("l",l)
#         print(s.graph)

        s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)



        ls_ = tf.multiply(l,s_)

        nls_ = tf.multiply(l,s_)*-1

        pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
#         print("nls",nls_)


    #     lst = tf.matmul(ls_,thetas)
    #     print("lst",lst)
        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
#         print("pout",pout)

#         print("t_pout",t_pout)

        t_k =  k*tf.squeeze(thetas)
#         print("t_k",t_k)

        zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))

        logz = tf.log(tf.reduce_sum(zy,axis=0))
#         print("zy",zy)
#         print("logz",logz)

        lsp = tf.reduce_logsumexp(t_pout)
#         print("lsp",lsp)

    #     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

        normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0) - logz))


#         print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

#         print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)
#         print("predict",predict)

    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(normloss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for it in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    while True:
                        _,ls = sess.run([train_step,normloss])
                        tl = tl + ls
                except tf.errors.OutOfRangeError:
                    pass
                print(it,"loss",tl)

                sess.run(dev_init_op)
                a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                print(a)
                print(t)


    #         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(gold_labels_dev,pl))

            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
            cf = confusion_matrix(gold_labels_dev,pl)
            print(cf)
            print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

In [64]:
## normalized training with penalty reduce_sum(max(0,-theta))

def train_nl_p(lr,ep,th):
    
    BATCH_SIZE = 1
    tf.reset_default_graph()


    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

        labels = tf.convert_to_tensor(gold_labels_dev)
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)

        next_element = iterator.get_next()
#         print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                                 dtype=tf.float64)

    #     thetas = tf.get_variable('thetas', [1,NoOfLFs],\
    #                              initializer=tf.truncated_normal_initializer(1,0.1,seed),\
    #                              dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

#         print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
#         print("k",k)
#         print(alphas.graph)
#         print(thetas.graph)
        l,s =  tf.unstack(next_element,axis=1)
#         print(alphas)
#         print(s)
#         print("l",l)
#         print(s.graph)

        s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)



        ls_ = tf.multiply(l,s_)

        nls_ = tf.multiply(l,s_)*-1

        pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
#         print("nls",nls_)


    #     lst = tf.matmul(ls_,thetas)
    #     print("lst",lst)
        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
#         print("pout",pout)

#         print("t_pout",t_pout)

        t_k =  k*tf.squeeze(thetas)
#         print("t_k",t_k)

        zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))

        logz = tf.log(tf.reduce_sum(zy,axis=0))
#         print("zy",zy)
#         print("logz",logz)

        lsp = tf.reduce_logsumexp(t_pout)
#         print("lsp",lsp)

    #     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

        normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0) - logz)) +\
                        tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas))


#         print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

#         print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)
#         print("predict",predict)

    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(normloss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for it in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    while True:
                        _,ls = sess.run([train_step,normloss])
                        tl = tl + ls
                except tf.errors.OutOfRangeError:
                    pass
                print(it,"loss",tl)

                sess.run(dev_init_op)
                a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                print(a)
                print(t)


    #         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(gold_labels_dev,pl))

            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
            cf = confusion_matrix(gold_labels_dev,pl)
            print(cf)
            print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

In [65]:
## normalized training with penalty2  -tf.minimum( tf.reduce_min(theta),0)

def train_nl_p2(lr,ep,th):
    
    BATCH_SIZE = 1
    tf.reset_default_graph()


    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

        labels = tf.convert_to_tensor(gold_labels_dev)
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)

        next_element = iterator.get_next()
        print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                                 dtype=tf.float64)

    #     thetas = tf.get_variable('thetas', [1,NoOfLFs],\
    #                              initializer=tf.truncated_normal_initializer(1,0.1,seed),\
    #                              dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

        print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
        print("k",k)
        print(alphas.graph)
        print(thetas.graph)
        l,s =  tf.unstack(next_element,axis=1)
        print(alphas)
        print(s)
        print("l",l)
        print(s.graph)

        s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)



        ls_ = tf.multiply(l,s_)

        nls_ = tf.multiply(l,s_)*-1

        pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
        print("nls",nls_)


    #     lst = tf.matmul(ls_,thetas)
    #     print("lst",lst)
        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
        print("pout",pout)

        print("t_pout",t_pout)

        t_k =  k*tf.squeeze(thetas)
        print("t_k",t_k)

        zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))

        logz = tf.log(tf.reduce_sum(zy,axis=0))
        print("zy",zy)
        print("logz",logz)

        lsp = tf.reduce_logsumexp(t_pout)
        print("lsp",lsp)

    #     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

        normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0) - logz)) \
                     -tf.minimum( tf.reduce_min(thetas),0.0)


        print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

        print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)
        print("predict",predict)

    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(normloss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for it in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    while True:
                        _,ls = sess.run([train_step,normloss])
                        tl = tl + ls
                except tf.errors.OutOfRangeError:
                    pass
                print(it,"loss",tl)

                sess.run(dev_init_op)
                a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                print(a)
                print(t)


    #         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(gold_labels_dev,pl))

            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
            cf = confusion_matrix(gold_labels_dev,pl)
            print(cf)
            print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

In [66]:
## normalized training with penalty3 sum(log(1+e^(-x-pk)))

def train_nl_p3(lr,ep,th,pk=0):
    
    BATCH_SIZE = 1
    tf.reset_default_graph()


    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

        labels = tf.convert_to_tensor(gold_labels_dev)
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)

        next_element = iterator.get_next()
        print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                                 dtype=tf.float64)

    #     thetas = tf.get_variable('thetas', [1,NoOfLFs],\
    #                              initializer=tf.truncated_normal_initializer(1,0.1,seed),\
    #                              dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

        print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
        print("k",k)
        print(alphas.graph)
        print(thetas.graph)
        l,s =  tf.unstack(next_element,axis=1)
        print(alphas)
        print(s)
        print("l",l)
        print(s.graph)

        s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)



        ls_ = tf.multiply(l,s_)

        nls_ = tf.multiply(l,s_)*-1

        pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
        print("nls",nls_)


    #     lst = tf.matmul(ls_,thetas)
    #     print("lst",lst)
        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
        print("pout",pout)

        print("t_pout",t_pout)

        t_k =  k*tf.squeeze(thetas)
        print("t_k",t_k)

        zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))

        logz = tf.log(tf.reduce_sum(zy,axis=0))
        print("zy",zy)
        print("logz",logz)

        lsp = tf.reduce_logsumexp(t_pout)
        print("lsp",lsp)

    #     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

        normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0) - logz)) \
                     +tf.reduce_sum(tf.log(1+tf.exp(-thetas-pk)))


        print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

        print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)
        print("predict",predict)

    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(normloss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for it in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    while True:
                        _,ls = sess.run([train_step,normloss])
                        tl = tl + ls
                except tf.errors.OutOfRangeError:
                    pass
                print(it,"loss",tl)

                sess.run(dev_init_op)
                a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                print(a)
                print(t)


    #         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(gold_labels_dev,pl))

            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
            cf = confusion_matrix(gold_labels_dev,pl)
            print(cf)
            print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

In [67]:
## normalized loss with prior from other LFs

def train_nlp(lr,ep,th):
    
    BATCH_SIZE = 1
    tf.reset_default_graph()


    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

        labels = tf.convert_to_tensor(gold_labels_dev)
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        
        
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)

        next_element = iterator.get_next()
        print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                                 dtype=tf.float64)

    #     thetas = tf.get_variable('thetas', [1,NoOfLFs],\
    #                              initializer=tf.truncated_normal_initializer(1,0.1,seed),\
    #                              dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,dtype=tf.float64)

        print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
        print("k",k)
        print(alphas.graph)
        print(thetas.graph)
        l,s =  tf.unstack(next_element,axis=1)
        print(alphas)
        print(s)
        print("l",l)
        print(s.graph)

        s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)

#         ls_ = tf.multiply(l,s_)

#         nls_ = tf.multiply(l,s_)*-1
               
        
        pout = tf.map_fn(lambda li: tf.map_fn(lambda lij:li*lij,li ),l)
#         print("nls",nls_)
        print("pout",pout)

    #     lst = tf.matmul(ls_,thetas)
    #     print("lst",lst)
        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
        

        print("t_pout",t_pout)

        t_k =  k*tf.squeeze(thetas)
        print("t_k",t_k)

        zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))

        logz = tf.log(tf.reduce_sum(zy,axis=0))
        print("zy",zy)
        print("logz",logz)

        

        sumy = tf.reduce_sum(t_pout-logz,axis=1)
        print("sumy",sumy)
    #     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

        normloss = tf.negative(tf.reduce_sum(tf.reduce_sum(t_pout-logz,axis=1) ))

        
        def index_along_every_row(array, index):
            N, _ = array.shape
            return array[np.arange(N), index]

        #Best LF
        blf = tf.argmax(t_pout,axis=1)
        print("blf",blf)
        print("normloss",normloss)
        
        
        marginals = tf.py_func(index_along_every_row, [tf.squeeze(t_pout), tf.squeeze(blf)], [tf.float64])[0]

        print("marginals",marginals)
        predict1 = tf.gather(k,tf.squeeze(blf))
        
        predict = tf.where(tf.equal(predict1,1),tf.ones_like(predict1),tf.zeros_like(predict1))
        print("predict",predict)

    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(normloss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for it in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    while True:
                        _,ls = sess.run([train_step,normloss])
                        tl = tl + ls
                except tf.errors.OutOfRangeError:
                    pass
                print(it,"loss",tl)

                sess.run(dev_init_op)
                a,t,m,pl,b = sess.run([alphas,thetas,marginals,predict,blf])
                print(b)
                print(t)


    #         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


                unique, counts = np.unique(b.tolist(), return_counts=True)
                print(dict(zip(unique, counts)))
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict,blf])
            print(b)
            print(t)

#             MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(gold_labels_dev,pl))

            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
            cf = confusion_matrix(gold_labels_dev,pl)
            print(cf)
            print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

In [370]:
#input L_S:train_L_S, K: no of classes

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def get_maj_prior(L_S,K):
    maj_prior = []
    
    print(L_S[:,0,:].shape)
    for row in np.nditer(L_S[:,0,:],flags=['external_loop'], order='C'):
        p = np.ones(K)/K
        unique, counts = np.unique(row, return_counts=True)
        unique = [int(x) for x in unique]
        rc = dict(zip(unique, counts))
        tnz = np.count_nonzero(row)
        if -1 in rc:
            p[0] = rc[-1]
        if 1 in rc:
            p[1] = rc[1]
        p = softmax(p)
        maj_prior.append(p)
    return np.array(maj_prior)

dev_maj_pl=get_maj_prior(dev_L_S,2)
print(dev_maj_pl.shape)


train_maj_pl=get_maj_prior(train_L_S,2)
print(train_maj_pl.shape)

(2814, 10)
(2814, 2)
(22276, 10)
(22276, 2)


In [73]:
## Normalized loss with majority prior



def train_nlmp(lr,ep,th):
    
    BATCH_SIZE = 1
    tf.reset_default_graph()


    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])
        
        maj_train_dataset = tf.data.Dataset.from_tensor_slices(train_maj_pl).batch(BATCH_SIZE)
        maj_dev_dataset = tf.data.Dataset.from_tensor_slices(dev_maj_pl).batch(dev_maj_pl.shape[0])

        labels = tf.convert_to_tensor(gold_labels_dev)
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        
        maj_iterator = tf.data.Iterator.from_structure(maj_train_dataset.output_types,
                                               maj_train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        maj_train_init_op = maj_iterator.make_initializer(maj_train_dataset)
       
        dev_init_op = iterator.make_initializer(dev_dataset)
        maj_dev_init_op = maj_iterator.make_initializer(maj_dev_dataset)

        next_element = iterator.get_next()
        maj_prior = tf.transpose(maj_iterator.get_next())
        print("maj_label",maj_prior)
        print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                                 dtype=tf.float64)

    #     thetas = tf.get_variable('thetas', [1,NoOfLFs],\
    #                              initializer=tf.truncated_normal_initializer(1,0.1,seed),\
    #                              dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

        print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
        print("k",k)
        print(alphas.graph)
        print(thetas.graph)
        l,s =  tf.unstack(next_element,axis=1)
        print(alphas)
        print(s)
        print("l",l)
        print(s.graph)

        s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)



        ls_ = tf.multiply(l,s_)

        nls_ = tf.multiply(l,s_)*-1

        pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
        print("nls",nls_)


    #     lst = tf.matmul(ls_,thetas)
    #     print("lst",lst)
        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
        print("pout",pout)

        print("t_pout",t_pout)

        t_k =  k*tf.squeeze(thetas)
        print("t_k",t_k)

        zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))

        logz = tf.log(tf.reduce_sum(zy,axis=0))
        print("zy",zy)
        print("logz",logz)
        stpout= tf.squeeze(t_pout)
        print("stpout",stpout)
        prod = tf.reduce_sum(maj_prior*stpout,axis=0)
        print("prod",prod)
     

    #     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

        normloss = tf.negative(tf.reduce_sum(tf.reduce_sum(maj_prior*tf.squeeze(t_pout-logz),axis=1) ))


        print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

        print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)
        print("predict",predict)

    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(normloss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for it in range(ep):
                sess.run(train_init_op)
                sess.run(maj_train_init_op)
                tl = 0
                try:
                    while True:
                        _,ls = sess.run([train_step,normloss])
                        tl = tl + ls
                except tf.errors.OutOfRangeError:
                    pass
                print(it,"loss",tl)

                sess.run(dev_init_op)
                sess.run(maj_dev_init_op)
                a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                print(a)
                print(t)


    #         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            sess.run(maj_dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(gold_labels_dev,pl))

            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
            cf = confusion_matrix(gold_labels_dev,pl)
            print(cf)
            print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

In [72]:
## Normalized loss with majority bias un-normalized

def train_unlmp(lr,ep,th):
    
    BATCH_SIZE = 1
    tf.reset_default_graph()


    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])
        
        maj_train_dataset = tf.data.Dataset.from_tensor_slices(train_maj_pl).batch(BATCH_SIZE)
        maj_dev_dataset = tf.data.Dataset.from_tensor_slices(dev_maj_pl).batch(dev_maj_pl.shape[0])

        labels = tf.convert_to_tensor(gold_labels_dev)
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        
        maj_iterator = tf.data.Iterator.from_structure(maj_train_dataset.output_types,
                                               maj_train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        maj_train_init_op = maj_iterator.make_initializer(maj_train_dataset)
       
        dev_init_op = iterator.make_initializer(dev_dataset)
        maj_dev_init_op = maj_iterator.make_initializer(maj_dev_dataset)

        next_element = iterator.get_next()
        maj_prior = tf.transpose(maj_iterator.get_next())
        print("maj_label",maj_prior)
        print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                                 dtype=tf.float64)

    #     thetas = tf.get_variable('thetas', [1,NoOfLFs],\
    #                              initializer=tf.truncated_normal_initializer(1,0.1,seed),\
    #                              dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

        print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
        print("k",k)
        print(alphas.graph)
        print(thetas.graph)
        l,s =  tf.unstack(next_element,axis=1)
        print(alphas)
        print(s)
        print("l",l)
        print(s.graph)

        s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)



        ls_ = tf.multiply(l,s_)

        nls_ = tf.multiply(l,s_)*-1

        pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
        print("nls",nls_)


    #     lst = tf.matmul(ls_,thetas)
    #     print("lst",lst)
        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
        print("pout",pout)

        print("t_pout",t_pout)

        t_k =  k*tf.squeeze(thetas)
        print("t_k",t_k)

        zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))

        logz = tf.log(tf.reduce_sum(zy,axis=0))
        print("zy",zy)
        print("logz",logz)
        stpout= tf.squeeze(t_pout)
        print("stpout",stpout)
        prod = tf.reduce_sum(maj_prior*stpout,axis=0)
        print("prod",prod)
     

    #     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

        normloss = tf.negative(tf.reduce_sum(tf.reduce_sum(maj_prior*tf.squeeze(t_pout),axis=1) ))


        print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

        print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)
        print("predict",predict)

    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(normloss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for it in range(ep):
                sess.run(train_init_op)
                sess.run(maj_train_init_op)
                tl = 0
                try:
                    while True:
                        _,ls = sess.run([train_step,normloss])
                        tl = tl + ls
                except tf.errors.OutOfRangeError:
                    pass
                print(it,"loss",tl)

                sess.run(dev_init_op)
                sess.run(maj_dev_init_op)
                a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                print(a)
                print(t)


    #         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            sess.run(maj_dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(gold_labels_dev,pl))

            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
            cf = confusion_matrix(gold_labels_dev,pl)
            print(cf)
            print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

In [71]:
## Un normalized training with different params

def train_unl(lr,ep,th):
    
    BATCH_SIZE = 1
    tf.reset_default_graph()


    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

        labels = tf.convert_to_tensor(gold_labels_dev)
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)

        next_element = iterator.get_next()
        print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                                 dtype=tf.float64)

    #     thetas = tf.get_variable('thetas', [1,NoOfLFs],\
    #                              initializer=tf.truncated_normal_initializer(1,0.1,seed),\
    #                              dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

        print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
        print("k",k)
        print(alphas.graph)
        print(thetas.graph)
        l,s =  tf.unstack(next_element,axis=1)
        print(alphas)
        print(s)
        print("l",l)
        print(s.graph)

        s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)



        ls_ = tf.multiply(l,s_)

        nls_ = tf.multiply(l,s_)*-1

        pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
        print("nls",nls_)


    #     lst = tf.matmul(ls_,thetas)
    #     print("lst",lst)
        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
        print("pout",pout)

        print("t_pout",t_pout)

        t_k =  k*tf.squeeze(thetas)
        print("t_k",t_k)

        zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))

        logz = tf.log(tf.reduce_sum(zy,axis=0))
        print("zy",zy)
        print("logz",logz)

        lsp = tf.reduce_logsumexp(t_pout)
        print("lsp",lsp)

    #     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

        normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)))


        print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

        print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)
        print("predict",predict)

    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(normloss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for it in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    while True:
                        _,ls = sess.run([train_step,normloss])
                        tl = tl + ls
                except tf.errors.OutOfRangeError:
                    pass
                print(it,"loss",tl)

                sess.run(dev_init_op)
                a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                print(a)
                print(t)


    #         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(gold_labels_dev,pl))

            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
            cf = confusion_matrix(gold_labels_dev,pl)
            print(cf)
            print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

In [85]:
# #stochastic + weighted cross entropy logits func + remove min(theta,0) in loss -- Marked
# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function
# import tensorflow as tf
# from tensorflow.contrib.tensorboard.plugins import projector

# def train_NN():
#     print()
#     result_dir = "./"
#     config = projector.ProjectorConfig()
#     tf.logging.set_verbosity(tf.logging.INFO)
#     summary_writer = tf.summary.FileWriter(result_dir)

#     tf.reset_default_graph()

#     dim = 2 #(labels,scores)

#     _x = tf.placeholder(tf.float64,shape=(dim,len(LFs)))

#     alphas = tf.get_variable('alpha', _x.get_shape()[-1],initializer=tf.constant_initializer(0.2),
#                             dtype=tf.float64)

#     thetas = tf.get_variable('theta', _x.get_shape()[-1],initializer=tf.constant_initializer(1),
#                             dtype=tf.float64)

#     l,s = tf.unstack(_x)

#     prelu_out_s = tf.maximum(tf.subtract(s,alphas), tf.zeros(shape=(len(LFs)),dtype=tf.float64))        

#     mul_L_S = tf.multiply(l,prelu_out_s)

#     phi_p1 = tf.reduce_sum(tf.multiply(mul_L_S,thetas))

#     phi_n1 = tf.reduce_sum(tf.multiply(tf.negative(mul_L_S),thetas))

#     phi_out = tf.stack([phi_n1,phi_p1])
    
#     predict = tf.argmax(tf.nn.softmax(phi_out))

#     loss = tf.negative(tf.reduce_logsumexp(phi_out))

#     train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss) 


#     check_op = tf.add_check_numerics_ops()

#     sess = tf.Session()
#     init = tf.global_variables_initializer()
#     sess.run(init)

#     for i in range(1):
#         c = 0
#         te_prev=1
#         total_te = 0
#         for L_S_i in train_L_S:

#             a,t,te_curr,_ = sess.run([alphas,thetas,loss,train_step],feed_dict={_x:L_S_i})
#             total_te+=te_curr

#             if(abs(te_curr-te_prev)<1e-200):
#                 break

#             if(c%4000==0):
#                 pl = []
#                 for L_S_i in dev_L_S:
#                     a,t,de_curr,p = sess.run([alphas,thetas,loss,predict],feed_dict={_x:L_S_i})
#                     pl.append(p)
#                 predicted_labels = [-1 if x==0 else x for x in pl]
#                 print()
#                 print(total_te/4000)
#                 total_te=0
# #                 print(a)
# #                 print(t)
# #                 print()
#                 print(predicted_labels.count(-1),predicted_labels.count(1))
#                 print(c," d ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='macro'))
#             c+=1
#             te_prev = te_curr
#         pl = []
#         for L_S_i in dev_L_S:
#             p = sess.run(predict,feed_dict={_x:L_S_i})
#             pl.append(p)
#         predicted_labels = [-1 if x==0 else x for x in pl]
#         print(i,total_te)
#         print(predicted_labels.count(-1),predicted_labels.count(1))
#         print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='macro'))
    
# train_NN()



-0.000226377142841
2251 545
0  d  (0.58865213829531426, 0.71341836734693875, 0.60408179957052133, None)

-1.94518369934e+28
2232 564
4000  d  (0.57825249752154351, 0.6933045525902668, 0.58885935866155448, None)

-5.04415736866e+58
2232 564
8000  d  (0.57825249752154351, 0.6933045525902668, 0.58885935866155448, None)

-1.33431810995e+89
2232 564
12000  d  (0.57825249752154351, 0.6933045525902668, 0.58885935866155448, None)

-3.67295517678e+119
2232 564
16000  d  (0.57825249752154351, 0.6933045525902668, 0.58885935866155448, None)

-9.52453175821e+149
2232 564
20000  d  (0.57825249752154351, 0.6933045525902668, 0.58885935866155448, None)
0 -1.71979948062e+170
2232 564
(0.57825249752154351, 0.6933045525902668, 0.58885935866155448, None)
